In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from classifier import Data

dataset = Data(
    data_path = "../data/imdb.train.csv",
    polarities = {
      "negative": 0,
      "positive": 1
    },
    data_label='review',
    target_label='sentiment'
)

In [4]:
from classifier._neural import Encoder

enc = Encoder({
    'model': 'bert-base-uncased',
    'layers': [11]
})

In [5]:
enc.df_encode(dataset.data, col=dataset.data_label, label=dataset.data_path)

In [6]:
import numpy as np
from sklearn.manifold import TSNE

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return TSNE(
            n_components=dim,
            learning_rate='auto',
            init='random'
        ).fit_transform(data)

In [7]:
import torch

prepared_data: np.ndarray = torch.stack(dataset.data[enc.col_name].tolist()).numpy()

In [8]:
reduced_data = manifold_reduction(prepared_data)
reduced_data.shape

(20170, 3)

In [9]:
dataset.data['reduced_embeds'] = list(reduced_data)

In [10]:
for _, group in dataset.data.groupby(dataset.target_label):
    group_data = np.stack(group['reduced_embeds'].tolist(), axis=0)
    print(group_data)
    print(np.mean(group_data, axis=0))

[[-15.43566    38.015152   -5.1018553]
 [-12.720923   11.365947   -2.1645703]
 [-19.392206   -5.0839853 -15.545003 ]
 ...
 [  4.6466174  21.00328    14.404374 ]
 [-16.219112   28.947287   18.520514 ]
 [  7.4869623  -9.957257  -30.68731  ]]
[-2.033787   -0.60669494 -1.8201021 ]
[[  7.556418   13.866396   23.80594  ]
 [  6.2049203   0.9103638 -13.070406 ]
 [  0.8778616  31.980783    6.574333 ]
 ...
 [ -4.6049824  25.121912   32.11588  ]
 [ 22.779156   -4.0274444  -8.603216 ]
 [  1.4513288  33.30259    16.543577 ]]
[1.2698871 0.8216684 1.7840573]
